$\text{\large Dimensionality Reduction} \\ 
\text{\small Author: Stellina Ao (Churchland Lab)} \newline
\text{\small Date Created: 07/24/2025} \newline
\text{\small Last Edited: 07/24/2025} \newline
\text{\small Purpose: Compare T-SNE plots between young and old mice to check for differences in clusters} 
$

Workflow
1. pd --> 2d gaussian normalization [v]
2. pca --> wvt []
3. bandpass the result of wavelet decomp []
4. tisne []

In [ ]:
# import packages
import numpy as np
import pandas as pd 
import pywt
import matplotlib.pyplot as plt
import plotly.express as px
import scienceplots

from scipy.stats import zscore, multivariate_normal
from scipy import signal
from sklearn.model_selection import ParameterGrid
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from lib import constants, data, pca_utils, wvt_utils, tsne_utils

import importlib

In [ ]:
# pretty plots
plt.style.use(['science','nature'])
plt.rcParams['figure.dpi'] = 200
%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}

## Import and Process Data

In [ ]:
importlib.reload(data)

In [ ]:
# import data and store in pd dataframe
dlc_outputs = data.read_data()
dlc_outputs = data.mean_imputation(dlc_outputs, 0.6) # impute the mean for uncertain vals (i.e., < 60% confidence) [must rectify by filtering out low freq bands intelligently]
dlc_outputs = data.norm_session_bp(dlc_outputs) # normalize per session and per body part
dlc_coords = data.get_coords(dlc_outputs)
# norm_dict = data.get_norm_dict(dlc_outputs) <- for normalization across animals

## PCA

In [ ]:
importlib.reload(pca_utils)

In [ ]:
# pca first!
p_ev = 0.9

# determine number of components to use <- 1 component should be good enough! (indicates huge correlation between the body parts)
evrs = pca_utils.get_pca_evr(dlc_coords, n_components=data.n_bodyparts*2, doPlot=True, n_rows=2, n_cols=4)
n_pcs, max_n_pcs = pca_utils.get_num_pcs(evrs, p_ev)
pca_utils.plot_cum_var(evrs, p_ev*100, n_pcs, n_rows=2, n_cols=4)

In [ ]:
n_components = max_n_pcs

dlc_reduxs = [PCA(n_components).fit_transform(dlc_coord) for dlc_coord in dlc_coords]

In [ ]:
pca_utils.plot_pca(dlc_reduxs, n_rows=2, n_cols=4)

## Wavelet Decomposition

In [ ]:
importlib.reload(wvt_utils)

In [ ]:
freqs_in = np.logspace(-4, 4, base=2)

In [ ]:
wvt_decomp_outputs = [wvt_utils.wavelet_decomp_pca(dlc_reduxs[i], freqs_in) for i in range(data.n_subj)]
cwtmatrs = [cwtmatr for (cwtmatr, freq) in wvt_decomp_outputs]
freqs = [freq for (cwtmatr, freq) in wvt_decomp_outputs]

In [ ]:
cwtmatrs_fvec = [np.reshape(np.transpose(cwtmatr), (cwtmatr.shape[2], cwtmatr.shape[0]*cwtmatr.shape[1])) for cwtmatr in cwtmatrs]

In [ ]:
n_freq = np.shape(freqs)[2]
n_fvec = n_components * (n_freq-1)

In [ ]:
pcms = wvt_utils.plot_wavelet_decomp_pca(cwtmatrs[0][:-1], freqs[0], n_rows=2, n_cols=5)

## T-SNE

In [ ]:
# TODO:
# * upload to repo and pull on mike's desktop
# * sanity check EVERYTHING (i.e., if i have to rerun this grid search because of some cs 2011 level bug i might combust)
# * run my (presumably) several day long grid search T-T

### TSNE Grid Search 

In [ ]:
models = []
embeds = []

param_grid = ParameterGrid({'perplexity': np.logspace(start=2, stop=6, base=2, num=5), 'early_exaggeration': np.logspace(start=0, stop=4, base=2, num=5), 'learning_rate': np.logspace(start=1, stop=4, base=10, num=4)})
max_iters = 5000
n_iters_without_progress = 500
n_jobs = 1

In [ ]:
from joblib import Parallel, delayed
def tsne(params, fvec):
    with open(f"tsne_gs_log.txt", "a") as f:
        print(f"running tsne with params {params}", file=f)

    tsne = TSNE(n_components=3, max_iter=max_iters, n_iter_without_progress=n_iters_without_progress, n_jobs=n_jobs, perplexity=params['perplexity'], learning_rate=params['learning_rate'], early_exaggeration=params['early_exaggeration'])
    embed = tsne.fit_transform(fvec)
    return (tsne, embed)
    
results = Parallel(n_jobs=16)(delayed(tsne)(params, fvec) for params, fvec in zip(param_grid, cwtmatrs_fvec))

In [ ]:
# * grid search with the perplexity, early exaggeration, learning rate, bump up max_iters and n_iters w/o converging (n_jobs = -1, i.e., take all my processors, almighty sklearn)

for params in param_grid:
    model_params = []
    embed_params = []
    for i, fvec in enumerate(cwtmatrs_fvec):
        tsne = TSNE(params)
        embed_params.append(tsne.fit_transform(fvec))
        model_params.append(tsne)
    models.append(model_params)
    embeds.append(embed_params)

In [ ]:
tsne_embed = [TSNE(n_components=3, perplexity=32).fit_transform(fvec) for fvec in cwtmatrs_fvec]

In [ ]:
tsne_flattened = np.array([coord for embed in tsne_embed for coord in embed])
idx = np.array([l for ls in [i*np.ones(len(embed)) for i, embed in enumerate(tsne_embed)] for l in ls]).astype('str')

In [ ]:
fig = px.scatter_3d(x=tsne_flattened[:,0], y=tsne_flattened[:,1], z=tsne_flattened[:,2], color=idx[:], title="T-SNE (All Subjects)")

fig.update_traces(marker=dict(size=0.5))

### K-Means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k_means = [KMeans().fit_predict(embed) for embed in tsne_embed]

In [ ]:
for i in range(len(tsne_embed)):
    fig = px.scatter_3d(x=tsne_embed[i][:,0], y=tsne_embed[i][:,1], z=tsne_embed[i][:,2], color=k_means[i].astype('str'), title=data.subject_ids[i])

    fig.update_traces(marker=dict(size=1))
    fig.show()

### Temporal Modulation

In [ ]:
for i in range(len(tsne_embed)):
    fig = px.scatter_3d(x=tsne_embed[i][:,0], y=tsne_embed[i][:,1], z=tsne_embed[i][:,2], color=np.arange(len(tsne_embed[i])), title=data.subject_ids[i])

    fig.update_traces(marker=dict(size=1))
    fig.update_layout(coloraxis_colorbar=dict(title="index"))
    fig.show()

### Probability Density

In [ ]:
importlib.reload(tsne_utils)

In [ ]:
tsne_utils.plot_density_all(tsne_embed)

In [ ]:
# plot probability density and watershed analysis
# ks test, plot cdf
# normalize or not
# check code

# increase freq range, one animal one body part, no pca

### Subsampling

In [ ]:
# importance sampling technique
# 1. generate an embedding using a selection of 600 data points from each subject out of 360k data points per subject [around 80k for us, 4.5 times less]
# 2. t-SNE on 20k randomly selected data points from each subject
# 3. embedding is used to estimate a probability density by convolving each point with a two-dimensional Gaussian whose width is equal to the distance from the point to its 10 nearest neighbours (N_embed)
# 4. this space is segmented by applying a watershed transform to the inverse of the pdf, creating a set of regions
# 5. finally, points are grouped by the region to which they belong and the number of points selected out of each region is proportional to the integral over the pdf in the region
# 6. repeat for all datasets, yielding 35k data points in the training set

def select_random_pts(data, n):
    n_points = len(data)
    
    if n > n_points:
        raise ValueError(f"ERROR: the number of points to sample {n} can't be greater than the size of the pool {n_points}")
    return np.random.choice(data, n)
    

In [ ]:
# grab 10k random data points from each subject and embed with tsne (step 2)

n_samples = 10000
cwtmatrs_10k_sample = np.reshape([fvec[select_random_pts(np.arange(len(fvec)), n_samples)] for fvec in cwtmatrs_fvec], (data.n_subj*n_samples, n_fvec))
tsne_10k = TSNE(n_components=3, perplexity=32).fit(cwtmatrs_10k_sample)

In [ ]:
tsne_10k_embed = TSNE(n_components=3, perplexity=32).fit_transform(cwtmatrs_10k_sample)

In [ ]:
px.scatter_3d(x=tsne_10k_embed[:10000,0], y=tsne_10k_embed[:10000,1], z=tsne_10k_embed[:10000,2])

In [ ]:
# embedding is used to estimate a probability density by convolving each point with a two-dimensional Gaussian whose width is equal to the distance from the point to its 10 nearest neighbours (N_embed) (step 3)
def gaussian_kernel(n, width, normalized=False):
    '''
    Generates a n x n matrix with a centered gaussian 
    of standard deviation std centered on it. If normalized,
    its volume equals 1.'''

    std = width / (2*(2*np.log(2))**0.5)
    gaussian1D = signal.gaussian(n, std)
    gaussian2D = np.outer(gaussian1D, gaussian1D)
    if normalized:
        gaussian2D /= (2*np.pi*(std**2))
    return gaussian2D



In [ ]:
tsne = TSNE(n_components=3, perplexity=32)
# then fit TSNE on the 35k sampled points and transform on every time point